## Import libraries

In [1]:
import pandas as pd
import lizard 


## Read data

In [2]:
data = pd.read_csv("./data/gh_apache_!_accumulo.csv")

In [3]:
data.head()


,project,url,discussion,pull_number,pull_id,filename,start_line,original_start_line,start_side,side,...,updated_at,commit_before,commit_while,commit_after,url_source_before,url_source_while,url_source_after,file_content_before,file_content_while,file_content_after
0,apache/accumulo,https://github.com/apache/accumulo/pull/1622,https://github.com/apache/accumulo/pull/1622#d...,1622,428010670,core/src/main/java/org/apache/accumulo/core/co...,NaN,NaN,NaN,RIGHT,...,2020-06-05T12:44:03Z,dbfab01a39e7bc6ac92f8641fc4b1c0f0c22039f,dbfab01a39e7bc6ac92f8641fc4b1c0f0c22039f,4d6991afc9c9589b829c2b9677a47e85586cc5aa,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...
1,apache/accumulo,https://github.com/apache/accumulo/pull/1622,https://github.com/apache/accumulo/pull/1622#d...,1622,428010670,core/src/main/java/org/apache/accumulo/core/co...,NaN,NaN,NaN,RIGHT,...,2020-06-05T12:44:03Z,dbfab01a39e7bc6ac92f8641fc4b1c0f0c22039f,dbfab01a39e7bc6ac92f8641fc4b1c0f0c22039f,4d6991afc9c9589b829c2b9677a47e85586cc5aa,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...
2,apache/accumulo,https://github.com/apache/accumulo/pull/1622,https://github.com/apache/accumulo/pull/1622#d...,1622,428010670,core/src/main/java/org/apache/accumulo/core/co...,NaN,NaN,NaN,RIGHT,...,2020-06-05T12:44:03Z,dbfab01a39e7bc6ac92f8641fc4b1c0f0c22039f,dbfab01a39e7bc6ac92f8641fc4b1c0f0c22039f,4d6991afc9c9589b829c2b9677a47e85586cc5aa,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...
3,apache/accumulo,https://github.com/apache/accumulo/pull/1622,https://github.com/apache/accumulo/pull/1622#d...,1622,428010670,core/src/main/java/org/apache/accumulo/core/co...,NaN,NaN,NaN,RIGHT,...,2020-06-05T12:44:03Z,dbfab01a39e7bc6ac92f8641fc4b1c0f0c22039f,dbfab01a39e7bc6ac92f8641fc4b1c0f0c22039f,4d6991afc9c9589b829c2b9677a47e85586cc5aa,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...
4,apache/accumulo,https://github.com/apache/accumulo/pull/1614,https://github.com/apache/accumulo/pull/1614#d...,1614,420303336,test/src/main/java/org/apache/accumulo/test/fu...,NaN,NaN,NaN,RIGHT,...,2020-06-04T19:21:57Z,13ad1b33bc3e92669db8a523e1e3a87025326d7a,ab92f60d0687aa9e3be8b0637240c088739b71fa,6319d9e408350cd15570eb9a93de613e5d4e730e,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...


## Data preprocessing

In [4]:
def extract_function_annotation(file_content, start_line, end_line):
    pass
    

In [5]:
def extract_function(file_content, start_line, end_line):
    """
    extract the function body from the file content
    """
    lines = file_content.splitlines()
    # -1 to exclude the last line }
    function_body = lines[start_line-1:end_line]
    # concat the lines
    function_body = " ".join(function_body)
    return function_body


In [6]:
def parse_functions(file_content_type):
    extracted_functions = []
    # loop through the rows. data is the dataframe that defined above
    for index, row in data.iterrows():
        file_content = row[file_content_type]
        # provide the file content to lizard to extract the functions, the function needs a filename to work so we just use the filename
        parse_result = lizard.analyze_file.analyze_source_code(row['filename'], file_content)
        functions = parse_result.function_list
        for function in functions:
            #print(function.__dict__)
            # from the file content, extract the function body using the start and end line
            method = extract_function(file_content, function.start_line, function.end_line)
            function_data = {
                "file_name": row['filename'],
                "dicussion": row['discussion'],
                "method_name": function.long_name,
                "method": method,
                "start_line": function.start_line,
                "end_line": function.end_line,
            }
            extracted_functions.append(function_data)
                    
    return extracted_functions




In [7]:
extracted_functions_while = parse_functions('file_content_while')
extracted_functions_after = parse_functions('file_content_after')


### Linking Reviewer Comments

In [8]:
# same file should'nt have the same method name more than once. Remove the duplicates of the same method name in the same file
def get_unique_methods(extracted_functions):
    extracted_functions = pd.DataFrame(extracted_functions)
    extracted_functions = extracted_functions.drop_duplicates(subset=['method_name', 'file_name'])
    extracted_functions = extracted_functions.reset_index(drop=True)
    return extracted_functions.to_dict('records')

In [9]:
unique_extracted_functions_while = get_unique_methods(extracted_functions_while)
unique_extracted_functions_after = get_unique_methods(extracted_functions_after)


In [15]:
# get the methods that we are in the file that we are interested in
def filter_methods(filename,is_while):
    if is_while:
        return [method for method in unique_extracted_functions_while if method['file_name'] == filename]
    else:
        return [method for method in unique_extracted_functions_after if method['file_name'] == filename]

In [11]:
# check if the function is in between the comment
def check_comment_in_line( commnet_start_line, comment_end_line, function_start_line, function_end_line):
    if function_start_line >= commnet_start_line and function_end_line <= comment_end_line:
        return True
    else:
        return False


In [12]:
# get the rows that owner_id and user_id is not same 
def filter_raw_dataframe(data):
    df = data[data['owner_id'] != data['user_id']]
    return df

In [16]:
# the purpose of this function is to link the comments with the methods that they are referring to
def link_comments_with_methods(is_while):
    df = filter_raw_dataframe(data)
    linked_methods = []
    linked_methods_data = {
        "method": {},
        "comment": {}
    }

    # loop through df dataframe 
    for index, row in df.iterrows():
        # if the comment start line is na that means the comment is a single line comment so we use the end line number as the start line number
        comment_start_line = row['original_start_line'] if not pd.isna(row['original_start_line']) else row['original_line']
        comment_end_line = row['original_line']
        filename = row['filename']
        filtered_methods = filter_methods(filename,is_while)
        for method in filtered_methods:
            # checm if the comment is in the method
            if check_comment_in_line(comment_start_line, comment_end_line, method['start_line'], method['end_line']):
                linked_methods_data['method'] = method
                linked_methods_data['comment'] = row
                linked_methods.append(linked_methods_data)
    return linked_methods
       


In [17]:
linked_methods_while = link_comments_with_methods(True)
linked_methods_after = link_comments_with_methods(False)
print(len(linked_methods_while))
print(len(linked_methods_after))

23
13
